# Recommendation System
### Using Matrix Factorization Embeddings and Deep Feed Forward Classifier

The Embeddings and Classifier will be trained using MovieLens 20M Dataset.

References:
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on Interactive Intelligent Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages. DOI=<http://dx.doi.org/10.1145/2827872>

Exploratory Data Analysis, Model Definition, Training and Fine-Tunning described step by step on this Notebook

In [0]:
from IPython.display import clear_output
!pip install --upgrade tensorflow-gpu
clear_output()

In [0]:
#Downloads and extract Dataset to local, wait for download, i dont want to put a progress bar here sorry
#You can run this on google colab for get faster downloads speeds
import os
import zipfile
import requests

resp = requests.get("http://files.grouplens.org/datasets/movielens/ml-20m.zip")

if(not os.path.exists("./Datasets/MoviLens20M.zip")):
  os.mkdir("./Datasets")

with open("./Datasets/MoviLens20M.zip", "wb") as f:
    f.write(resp.content)

with zipfile.ZipFile("./Datasets/MoviLens20M.zip", "r") as zip_ref:
  zip_ref.extractall("./Datasets")

In [0]:
#Imports
import tensorflow as tf
import pandas as pd
import numpy as np

In [13]:
#Loads Dataset, we only need ratings.csv file
ml_df = pd.read_csv("./Datasets/ml-20m/ratings.csv")
ml_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [14]:
#Check for null values
ml_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64